<a href="https://colab.research.google.com/github/Jass1313c/comp215/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime, json, requests
from pprint import pprint

param = { "format": "geojson", "starttime": "2024-03-10","limit": 20000, "endtime": "2024-05-30"}
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
response = requests.get(url, params=param)
data = json.loads(response.text)

pprint(data)

for feature in data['features']:
  depth = feature['geometry']['coordinates'][2]
  if depth < 10 and "Texas" in feature['properties']['place']:
    Latitude = feature['geometry']['coordinates'][1]
    Longitude = feature['geometry']['coordinates'][0]
    print(f"Mag: {feature['properties']['mag']},   Place: {feature['properties']['place']},   Time: {datetime.datetime.fromtimestamp(feature['properties']['time'] / 1000)},   Lat: {Latitude},   Long: {Longitude}")

In [8]:
import datetime, json, requests
from pprint import pprint
import pyproj

param = { "format": "geojson", "starttime": "2024-03-14","limit": 20000, "endtime": "2024-06-15", "minmagnitude": 1}
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
response = requests.get(url, params=param)
data = json.loads(response.text)

#Convert coordinate systems of fracking data to WGS84..
source_crs = 'epsg:4269'  # NAD83
target_crs = 'epsg:4326'  # WGS84

transformer = pyproj.Transformer.from_crs(source_crs, target_crs)

count = 0
earthquake_data = []


for feature in data['features']:
  depth = feature['geometry']['coordinates'][2]
  if depth < 10 and "Texas" in feature['properties']['place']:
    count += 1

    time = datetime.datetime.fromtimestamp(feature['properties']['time'] / 1000)

    # Transform coordinates
    lat_wgs84, long_wgs84 = transformer.transform(feature['geometry']['coordinates'][1], feature['geometry']['coordinates'][0])

    earthquake_data.append({
      "Mag": feature['properties']['mag'],
      "Place": feature['properties']['place'],
      "Time": time,
      "Lat": lat_wgs84,
      "Long": long_wgs84,
      "Depth": depth
    })

pprint(earthquake_data)
pprint(f"Count: {count}")

[{'Depth': 3.9722,
  'Lat': 31.034,
  'Long': -103.621,
  'Mag': 1.6,
  'Place': '13 km ENE of Balmorhea, Texas',
  'Time': datetime.datetime(2024, 6, 14, 23, 58, 49, 880000)},
 {'Depth': 3.7842,
  'Lat': 32.339,
  'Long': -101.521,
  'Mag': 1.1,
  'Place': '10 km NNW of Big Spring, Texas',
  'Time': datetime.datetime(2024, 6, 14, 22, 51, 20, 459000)},
 {'Depth': 7.9968,
  'Lat': 31.706,
  'Long': -104.16,
  'Mag': 2,
  'Place': '53 km W of Mentone, Texas',
  'Time': datetime.datetime(2024, 6, 14, 22, 47, 37, 676000)},
 {'Depth': 6.2537,
  'Lat': 31.666,
  'Long': -104.187,
  'Mag': 2,
  'Place': '53 km NW of Toyah, Texas',
  'Time': datetime.datetime(2024, 6, 14, 22, 45, 16, 961000)},
 {'Depth': 7.843,
  'Lat': 31.703,
  'Long': -104.165,
  'Mag': 1.7,
  'Place': '53 km W of Mentone, Texas',
  'Time': datetime.datetime(2024, 6, 14, 22, 37, 47, 421000)},
 {'Depth': 6.8945,
  'Lat': 31.701,
  'Long': -104.16,
  'Mag': 1.7,
  'Place': '53 km W of Mentone, Texas',
  'Time': datetime.datet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Fracking.csv'
fracking_df = pd.read_csv(file_path)
with open(file_path, 'r') as file:
    fracking_df = pd.read_csv(file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from geopy.distance import geodesic

# Load fracking data
file_path = '/content/drive/MyDrive/Fracking.csv'
fracking_df = pd.read_csv(file_path)

# Convert JobStartDate & JobEndDate to datetime
fracking_df["JobStartDate"] = pd.to_datetime(fracking_df["JobStartDate"])
fracking_df["JobEndDate"] = pd.to_datetime(fracking_df["JobEndDate"])

# Create a DataFrame for earthquake data
df_quakes = pd.DataFrame([
    {'Depth': 6.7871, 'Lat': 32.43, 'Long': -102.144, 'Mag': 1.4, 'Place': '38 km SSW of Los Ybanez, Texas', 'Time': datetime.datetime(2024, 6, 12, 17, 54, 44, 924000)},
    {'Depth': 6.5186, 'Lat': 28.494, 'Long': -98.631, 'Mag': 2.0, 'Place': '8 km WNW of Tilden, Texas', 'Time': datetime.datetime(2024, 6, 12, 17, 16, 50, 386000)},
    {'Depth': 5.9332, 'Lat': 31.647, 'Long': -104.02, 'Mag': 1.6, 'Place': '40 km W of Mentone, Texas', 'Time': datetime.datetime(2024, 6, 12, 17, 7, 30, 664000)},
    {'Depth': 9.5459, 'Lat': 32.409, 'Long': -102.089, 'Mag': 1.3, 'Place': '37 km WSW of Ackerly, Texas', 'Time': datetime.datetime(2024, 6, 12, 16, 58, 8, 809000)},
    {'Depth': 7.3944, 'Lat': 31.704, 'Long': -104.159, 'Mag': 1.8, 'Place': '53 km W of Mentone, Texas', 'Time': datetime.datetime(2024, 6, 12, 16, 34, 51, 955000)},
    {'Depth': 8.2019, 'Lat': 31.512, 'Long': -104.015, 'Mag': 2.4, 'Place': '30 km NW of Toyah, Texas', 'Time': datetime.datetime(2024, 6, 12, 3, 25, 17, 553000)},
    {'Depth': 5.7861, 'Lat': 32.433, 'Long': -102.139, 'Mag': 1.4, 'Place': '37 km SSW of Los Ybanez, Texas', 'Time': datetime.datetime(2024, 6, 12, 2, 25, 7, 588000)}
])

# Iterate over earthquake data and check for matches
matched_data = []

for _, quake in df_quakes.iterrows():
    quake_time = quake["Time"]
    quake_location = (quake["Lat"], quake["Long"])

    for _, frack in fracking_df.iterrows():
        frack_location = (frack["Latitude"], frack["Longitude"])

        # Check if earthquake is within 10 km of fracking site
        if geodesic(quake_location, frack_location).km <= 10:
            # Check if earthquake happened during or after fracking job
            if frack["JobStartDate"] <= quake_time <= frack["JobEndDate"]:
                matched_data.append({
                    "Frack_Well": frack["DisclosureId"],
                    "Frack_Start": frack["JobStartDate"],
                    "Frack_End": frack["JobEndDate"],
                    "Earthquake_Time": quake_time,
                    "Mag": quake["Mag"],
                    "Depth": quake["Depth"],
                    "Distance_km": geodesic(quake_location, frack_location).km
                })

# Convert matched results to DataFrame
matched_df = pd.DataFrame(matched_data)

# Save to CSV for further analysis
matched_df.to_csv("/mnt/data/Matched_Earthquake_Fracking.csv", index=False)

print(f"Matched events: {len(matched_df)}")
print(matched_df.head())


<ipython-input-37-cc26aea9b426>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fracking_df["JobStartDate"] = pd.to_datetime(fracking_df["JobStartDate"])
<ipython-input-37-cc26aea9b426>:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fracking_df["JobEndDate"] = pd.to_datetime(fracking_df["JobEndDate"])


OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [ ]:
from geopy import distance
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(distance.distance(newport_ri, cleveland_oh).km)

866.4554329098685


In [ ]:
import pyproj
import pandas as pd

# Define source projection (NAD 83)
source_proj = pyproj.Proj('epsg:4269')

# Define target projection (WGS 84)
target_proj = pyproj.Proj('epsg:4326')

# Assuming you have latitude and longitude as individual variables
# Create a DataFrame
df = pd.DataFrame({'Latitude': [Latitude], 'Longitude': [Longitude]})

# Create new columns for projected coordinates
df['Latitude_WGS84'], df['Longitude_WGS84'] = pyproj.transform(source_proj, target_proj, df['Longitude'].values, df['Latitude'].values)

print(df)

   Latitude  Longitude  Latitude_WGS84  Longitude_WGS84
0    31.014   -103.254        -103.254           31.014


<ipython-input-16-2e97aa9c9552>:15: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  df['Latitude_WGS84'], df['Longitude_WGS84'] = pyproj.transform(source_proj, target_proj, df['Longitude'].values, df['Latitude'].values)


In [6]:
import pyproj

# Define the source and target coordinate systems
source_crs = 'epsg:4269'  # NAD83
target_crs = 'epsg:4326'  # WGS84

# Create a transformer object
transformer = pyproj.Transformer.from_crs(source_crs, target_crs)

# Define the input coordinates
latitude = 31.2137
longitude = -103.4621

# Perform the transformation
latitude_wgs84, longitude_wgs84 = transformer.transform(latitude, longitude)

# Print the transformed coordinates
print(f"WGS84 Latitude: {latitude_wgs84}")
print(f"WGS84 Longitude: {longitude_wgs84}")

WGS84 Latitude: 31.2137
WGS84 Longitude: -103.4621
